# РАЗРАБОТКА МОДЕЛИ PREF2VEC

## 1) Импорт используемых библиотек

Сначала необходимо инпортировать все библиотеки, которые будут использоваться при создании модели.

Для работы с содержимым используемого Google Диска через платформу Google Colab требуется импортировать следующие библиотеки:
- `drive` - модуль, который позволяет подключить Google Диск к виртуальной машине среды выполнения и использовать его содержимое.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Далее импортируем библиотеки, используемые в дальнейшем:
- `pandas` - библиотека для обработки и анализа структурированных данных;
- `numpy` - библиотека, которую применяют для математических вычислений: начиная с базовых функций и заканчивая линейной алгеброй;
- `re` - библиотека, предоставляющая мощные инструменты для работы с текстом.
- `gensim` - библиотека обработки естественного языка предназначения для «Тематического моделирования»;
- `sklearn` - библиотека, реализующая методы машинного обучения, в состав которой входят различные алгоритмы, в том числе предназначенные для задач классификации, регрессионного и кластерного анализа данных, включая метод опорных векторов, метод случайного леса, алгоритм усиления градиента, метод k-средних и DBSCAN;
- `plotly` - графическая библиотека, которая позволяется создавать интерактивные графики.;
- `wordcloud` - библиотека, с помощью которой реализуется метод визуализации данных облако слов, используемый для представления текстовых данных, в котором размер каждого слова указывает на его частоту или важность.

In [12]:
pip install scikit-learn-extra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.7 MB/s eta 0:00:00


In [14]:
import numpy as np
import pandas as pd

import re
import collections
import datetime

from gensim.parsing.preprocessing import preprocess_string
from gensim.models.doc2vec import Doc2Vec

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn_extra.cluster import KMedoids
from scipy.spatial import distance

import plotly.express as px
from plotly.subplots import make_subplots
from wordcloud import WordCloud

## 2) Подготовка используемых ресурсов

Определим корпус книжных аннотаций и датасет пользовательских предпочтений, которые будут использованы далее при разработке и анализе работы:

In [ ]:
df_user_data = pd.read_csv(
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_user_data.csv',
    index_col='user_id'
)
df_corpus_annotations = pd.read_csv(
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_corpus_annotations.csv',
    index_col='book_id'
)

После определим ранее обученную модель Doc2Vec:

In [ ]:
filename_d2v = '/content/drive/MyDrive/ВКР/children/children_model_d2v.d2v'
model_d2v = Doc2Vec.load(filename_d2v)

VECTOR_SIZE = model_d2v.vector_size
print(f"Размерность формируемых векторов - {VECTOR_SIZE}")

Размерность формируемых векторов - 200


Также определим ранее разработанный класс `UES`:

In [13]:
!python /content/drive/MyDrive/ВКР/user_embeddings_space.py

## 3) Разработка класса Pref2Vec

При помощи класса UES появляется возможность векторизации всех взаимодействий пользователя с объектами – построение пользовательского пространства эмбеддингов. Рассчитав пространства эмбеддингов для ряда пользователей, открывается перспектива исследования схожих пространств и построение рекомендаций на основе полученных данных.

Приведём код разработанной модели `Pref2Vec`, которая реализовывает исследование пользовательских пространств эмбеддингов:

In [ ]:
class Pref2Vec(object):
    '''
    PREF2VEC

    Модель Pref2Vec - модель векторизации пользовательских предпочтений Pref2Vec.
    Модель позволяет построить множество пользовательских пространств эмбеддингов
    – корпус объектов UES, представленных в виде набора вычисленных центров
    кластеров взаимодействий. На основе сформированного корпуса открывается
    возможность исследования пользовательских пространств эмбеддингов, которое
    позволяет строить рекомендации для конечного пользователя посредством поиска
    схожих пользовательских предпочтений, то есть схожих пространств. Дальнейшая
    рекомендация объектов происходит при помощи анализа расширенных эмбеддингов
    наиболее подобных пользовательских пространств.
    '''


    def __init__(self, depth):
        '''
        Метод инициализации класса, в котором производится определение полей
        `self.depth` и `self.corpus`.

        Аргументы:
        - depth (`int`) -
        '''

        self.depth = depth
        self.corpus = pd.DataFrame(
            columns=['user_id', 'center_read', 'center_shelved',
                     'center_rating_0', 'center_rating_1', 'center_rating_2',
                     'center_rating_3', 'center_rating_4', 'center_rating_5']
        )
        target_interactions = ['read', 'rating_4', 'rating_5']


    def build_corpus(self, filepath_or_buffer, users_data):
        '''
        Метод, при помощи которого производится построение корпуса
        пользовательских предпочтений.

        Для каждого пользователя рассчитывается пространство эмбеддингов (UES),
        далее полученные центры кластеров взаимодействий сохраняются в корпус
        модели.

        Аргументы:
        - users_data (`pandas.DataFrame`)
        '''

        counter = 0
        for index, row in users_data.iterrows():
            # Расчёт проистранства эмбеддингов
            ues = UES(row)
            # Получение центров кластаров взаимодействий
            centers = ues.retrieve_centers()
            # Добавление данных в корпус
            new_row = {'user_id': ues.user_id}
            for interaction in interactions:
                new_row[f"center_{interaction}"] = self.__get_element_or_nan(
                    array=centers[centers['interaction'] == interaction]['embedding'].values,
                    index=0
                )
            self.corpus.loc[len(self.corpus)] = new_row

            counter += 1
            if (counter % 250 == 0):
                self.__save_corpus(filepath_or_buffer)

        self.corpus = self.corpus.set_index('user_id')
        self.__save_corpus(filepath_or_buffer)


    def __save_corpus(self, filepath_or_buffer):
        '''
        Метод, который сохраняет корпус модели в CSV-файл.

        Аргументы:
        - filepath_or_buffer (`str`) - путь файла, в который произойдёт
            сохранение файла.
        '''

        self.corpus.to_csv(filepath_or_buffer)


    def load_corpus(self, filepath_or_buffer):
        '''
        Метод, позволяющий загрузить корпус модели из CSV-файла.

        Аргументы:
        - filepath_or_buffer (`str`) - путь к используемому CSV-файлу.
        '''

        users_data = pd.read_csv(
            filepath_or_buffer=filepath_or_buffer,
            index_col='user_id'
        )
        # Преобразование строковых значений датасета в значения с плавающей запятой
        for user_id in users_data.index.values:
            for interaction in interactions:
                try:
                    if not (pd.isna(users_data.loc[user_id][f"center_{interaction}"])):
                        center = np.asarray(
                            np.array(re.findall('[-+0-9.e]+', users_data.loc[user_id][f"center_{interaction}"])),
                            dtype=float
                        )
                        users_data.at[user_id, f"center_{interaction}"] = center
                except:
                    continue
        self.corpus = users_data


    def most_similar_users(self, target_user_data, topn=10, show_info=True):
        '''
        Метод, при помощи которого происходит поиск ближайших `topn`
        пользователей к целевому поиску.

        Поиск осуществляется рассчётом косинусных расстоний между центрами
        кластеров категорий искомого пользователя и соответствующими центрами
        пользователей корпуса модели `self.corpus`. Для каждой из категорий
        происходит рассчёт ближайших `self.depth` пользователей. Далее выводятся
        самые близкие пользователи к целевому.

        Аргументы:
        - target_user_data (`pandas.Series`) - серия библиотеки Pandas, в которой
            содержится препроцессированная информация о взаимодействиях
            целевого пользователя с книгами в виде категоризарованного набора
            идентификаторов книг;
        - topn (`int`) - количество ближайших пользователей;
        - show_info (`bool`) - флаг вывода информации о работе метода.

        Возвращает:
        - most_similar_users (`list`) - список с ближашими пользователями.
            Содержит набор объектов `dict`, в которых хранятся идентификатор
            пользователя, категория в которой пользователь оказался близок и
            косинусное расстояние с центром категории целевого пользователя.
        '''

        # Датасет поиска ближайших пользователей
        self.df_most_similar_users = pd.DataFrame(
            columns=['user_id', 'interaction', 'interaction_index', 'similarity']
        )
        # Рассчёт целевого пользовательского пространтва эмбеддингов
        target_ues = UES(target_user_data)
        target_user_id = target_ues.user_id
        target_centers = target_ues.retrieve_centers()

        # Поиск ближайших центров по видам взаимодействий
        for interaction in interactions:
            self.__most_similar_users_by_interaction(
                target_user_id=target_user_id,
                target_center=target_centers[target_centers['interaction'] == interaction]['embedding'].values,
                interaction=interaction
            )
        # Группировка и агрегирование полученный данных
        df_most_similar_users_aggregated = self.df_most_similar_users.groupby('user_id').agg({'interaction': 'count', 'interaction_index': 'mean'})
        df_most_similar_user_top_n = df_most_similar_users_aggregated.sort_values(by=['interaction', 'interaction_index'], ascending=[False, True]).head(topn)
        most_similar_users = list()
        for user_id in df_most_similar_user_top_n.index.values:
            df_current_user = self.df_most_similar_users[self.df_most_similar_users['user_id'] == user_id]
            dict_current_user = {'user_id': user_id, 'appearances': []}
            for index, row in df_current_user.iterrows():
                dict_current_user['appearances'].append((row['interaction'], row['similarity']))
            most_similar_users.append(dict_current_user)

        # Вывод результатов
        if (show_info):
            print(f"Ближайщие пользователи к {target_user_id}:")
            for i in range(len(most_similar_users)):
                print(f"{i+1}) Пользователь №{most_similar_users[i]['user_id']}")
                for appearance in most_similar_users[i]['appearances']:
                    print(f"   {appearance[0]} - {appearance[1]}")

        return most_similar_users


    def __most_similar_users_by_interaction(self, target_user_id, target_center, interaction):
        '''
        Метод, осуществляющий рассчёт косинусных расстоний между искомым центром
        категории и всеми остальными центрами категории корпуса `self.corpus`.

        Расстоние вычисляется при помощи модуля `metrics.pairwise.cosine_similarity`
        библиотеки `sklearn`. Для увеличения скорости вычислений промежуточные
        результаты сохраняются и обрабатываются при помощи библиотеки `pandas`.

        Аргументы:
        - target_user_id (`str`) - идентификатор целевого пользователя;
        - target_center (`list`) - массив, содержащий целевой центр кластера
            текущего взаимодействия;
        - interaction ('str') - рассматриваемое взаимодействие.
        '''

        # Если у пользователя нет эмбеддингов текущего взаимодействия, ближайщих пользователей нет
        if (target_center.size == 0):
            return

        # Инициализация датасета, в котором будут рассчитываться расстония
        df_similarity = pd.DataFrame(data={'user_id': self.corpus.index.values, 'center': self.corpus[f"center_{interaction}"]})
        # Фильтрация элементов, в которых нет центра рассматриваемой категории
        df_similarity = df_similarity[df_similarity['center'].notna()]
        # Вычисление расстояний между искомым центром и всеми остальными
        target_center = [target_center[0]] if (target_center.size > 1) else list(target_center)
        similarities = cosine_similarity(target_center, list(df_similarity['center']))
        df_similarity['similarity'] = np.reshape(similarities, df_similarity['center'].size)
        # Обработка случая, в котором найденное рассторие - расстояние с искомым центром
        if (df_similarity['similarity'].idxmax() == target_user_id):
            df_similarity = df_similarity.drop(df_similarity['similarity'].idxmax())

        # Сохранение ближайших пользователей в датасет self.df_most_similar_users
        df_most_similar_users = pd.DataFrame(
            data={
                'user_id': df_similarity['similarity'].nlargest(self.depth).index.values,
                'interaction': [interaction] * self.depth,
                'interaction_index': [i+1 for i in range(self.depth)],
                'similarity': df_similarity['similarity'].nlargest(self.depth).values
            }
        )
        self.df_most_similar_users = pd.concat([self.df_most_similar_users, df_most_similar_users])


    def recommend_for_user(self, target_user_data, topn=50, show_info=True):
        '''
        Метод, при помощи которого происходит рассчёт `topn` рекомендаций для
        целевого пользователя.

        Сначала рассчитываются 10 ближайших пользователей к целевому. Далее
        рассматриваются все объекты ближащий пользователей на предмет
        косинусного расстояния к объектам целевого пользоваля - происходит
        рассчёт расстояний между рассматриваемым объектом и всеми целевыми,
        наименьшее расстояние сохраняется. После происходит сортировка объектов
        по полученным расстояниям и вывод `topn` ближайших.

        Аргументы:
        - target_user_data (`pandas.Series`) - серия библиотеки Pandas, в которой
            содержится препроцессированная информация о взаимодействиях
            целевого пользователя с книгами в виде категоризарованного набора
            идентификаторов книг;
        - topn (`int`) - количество ближайших пользователей;
        - show_info (`bool`) - флаг вывода информации о работе метода.

        Возвращает:
        - recommendations (`list`) - список с полученными рекомендациями.
            Содержит пары с идентификатором рекомендуемого объекта и ближайщим
            расстоянием.
        '''

        # Рассчёт целевого пользовательского пространтва эмбеддингов
        ues_target = UES(target_user_data)
        ues_target.df = ues_target.df[ues_target.df['interaction'].isin(target_interactions)]
        # Датасет, в котором будут рассчитываться косинусные расстояния
        self.df_embeggings_target = pd.DataFrame(
            data={'embedding': ues_target.df['embedding'].values},
            index=[re.findall('[0-9][0-9]+', index)[0] for index in ues_target.df.index.values]
        )

        # Поиск ближайших пользователей
        most_similar_users = self.most_similar_users(target_user_data, show_info=False)
        most_similar_users_ids = [x['user_id'] for x in most_similar_users]
        # Датасет с расстояниями всех объектов с искомыми
        self.df_recommendations = pd.DataFrame(columns=['book_id', 'similarity'])
        # Прогон объектов всех ближайших пользователей
        for user_id in most_similar_users_ids:
            ues_near = UES(df_user_data.loc[user_id])
            ues_near.df = ues_near.df[ues_near.df['interaction'].isin(target_interactions)]
            # Вычисление ближайшего расстония для каждого объекта с целевыми объектами
            for index, embedding in ues_near.df['embedding'].items():
                nearest_similarity = self.__get_nearest_similarity(
                    embedding=embedding,
                    embedding_id=re.findall('[0-9][0-9]+', index)[0]
                )
                self.df_recommendations.loc[len(self.df_recommendations.index)] = {
                    'book_id': re.findall('[0-9][0-9]+', index)[0],
                    'similarity': nearest_similarity
                }

        # Сортировка объектов и вывод `topn` ближайших
        self.df_recommendations = self.df_recommendations.drop_duplicates('book_id', keep='first')
        self.df_recommendations = self.df_recommendations.sort_values(by='similarity', ascending=False).head(topn)
        recommendations = []
        for index, row in self.df_recommendations.iterrows():
            recommendations.append((row['book_id'], row['similarity']))

        # Вывод результатов
        if (show_info):
            print(f"Рекомендации для пользователя {ues_target.user_id}:")
            counter = 1
            for index, row in self.df_recommendations.iterrows():
                print(f"{counter}) Книга #{row['book_id']} - {row['similarity']}")
                counter += 1

        return recommendations


    def __get_nearest_similarity(self, embedding, embedding_id):
        '''
        Метод, который позволяет найти ближайшее косинусное расстояние к одному
        из целевых объектов датасета `self.df_embeggings_target`.

        Аргументы:
        - embedding (`list`) - эмбеддинг расссматриваемого объекта;
        - embedding_id (`str`) - идентификатор расссматриваемого объекта.
        '''

        # Рассчёт растояний между рассматриваемым объектом и целевыми
        similarities = cosine_similarity([embedding], list(self.df_embeggings_target['embedding']))
        self.df_embeggings_target['similarity'] = np.reshape(similarities, self.df_embeggings_target['embedding'].size)

        # Обработка случая, в котором найденное рассторие - расстояние с искомым объектом
        if (self.df_embeggings_target['similarity'].idxmax() == embedding_id):
            self.df_embeggings_target.drop(self.df_embeggings_target['similarity'].idxmax())

        return self.df_embeggings_target['similarity'].max()


    def __get_element_or_nan(self, array, index):
        '''
        Метод, который возвращает элемент массива, если его адрес находится
        в пределах массива. В ином случае возвращается элемент `np.nan`.

        Аргументы:
        - array (`array-like`) - массив, из которого необходимо получить элемент;
        - index (`int`) - индекс получаемого элемента.
        '''

        try:
            return array[index]
        except IndexError:
            return np.nan


## 4) Рассчёт корпуса пользовательских пространств

Сперва требуется произвести построение корпуса модели: рассчитать центры кластеров взаимодействий для каждого из предоставляемых пользовательских пространств.

In [ ]:
%%time

model_p2v = Pref2Vec(depth=500)
model_p2v.build_corpus(
    users_data=df_user_data.iloc[90000:100000],
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_corpus_pref2vec_13.csv'
)

CPU times: user 4h 49min 24s, sys: 2min 47s, total: 4h 52min 11s
Wall time: 4h 53min 46s


In [16]:
corpus_pref2vec = pd.read_csv(
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_corpus_pref2vec.csv',
    index_col='user_id'
)
corpus_pref2vec.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 8842281e1d1347389f2ab93d60773d4d to e9104aee79c530cef254ee043bbfc859
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   center_read      89818 non-null  object
 1   center_shelved   65270 non-null  object
 2   center_rating_0  15005 non-null  object
 3   center_rating_1  6972 non-null   object
 4   center_rating_2  19169 non-null  object
 5   center_rating_3  46692 non-null  object
 6   center_rating_4  63323 non-null  object
 7   center_rating_5  69363 non-null  object
dtypes: object(8)
memory usage: 6.9+ MB


Также имеется опция загрузки ранее рассчитанного корпуса:

In [ ]:
filename_corpus = '/content/drive/MyDrive/ВКР/Промежуточные результаты/corpus_pref2vec_5.csv'

model_p2v = Pref2Vec(depth=100)
model_p2v.load_corpus(filename_corpus)

model_p2v.corpus.head()

,center_read,center_shelved,center_rating_0,center_rating_1,center_rating_2,center_rating_3,center_rating_4,center_rating_5
user_id,,,,,,,,
8fcba824400e40636aea29f2b5594194,"[0.00136383623, -0.00219704513, 0.00100740441,...",NaN,NaN,NaN,NaN,NaN,"[-0.00752301142, 0.0101357624, 0.00427832082, ...",NaN
2dec0d0c33146069e5bdc438d86bb6cf,"[-0.00457797805, -0.0017531293, 0.00196525757,...","[-0.00591568695, 0.00323694386, 0.00439793244,...",NaN,NaN,NaN,NaN,NaN,"[0.00205222331, -0.00900319591, -0.00014116983..."
ad133fa9fed1527fbf4e6bf1afff495e,"[-0.00532039069, -0.0102215586, -0.00909158308...",NaN,NaN,NaN,NaN,"[0.000819509383, -0.00265661906, 0.00113685196...","[0.0042772973, 0.00145656662, -0.00451587234, ...",NaN
efb0b7d931b8b6ccd8d40d0a18b76292,"[0.0015029012, 0.000641715247, -0.000823338516...","[0.00843419041, -0.00966246519, -0.0105564902,...","[0.0217126198, -0.000335702294, -0.0302965883,...","[0.00857563, -0.00960512, -0.01591355, 0.00784...","[0.0113355983, 0.0315268449, -0.00718603702, 0...","[-0.00104464963, -0.00340407947, 0.00134930655...","[-0.000643946754, -0.00057956489, -5.84888994e...","[-0.00232150522, -0.00328354351, 0.00416622683..."
7f6dac726b59fb3d6662667e91010d16,"[0.0038297784, -0.00291902153, -0.000408341031...","[-0.00387233472, -0.00384449027, 0.00064547587...",NaN,NaN,NaN,NaN,"[0.000487339188, -0.00198634667, -0.0025479751...",NaN


## 5) Поиск ближайших пользователей

После построения или загрузки корпуса открывается возможность поиска ближайших пользовательских пространств к целевому.

Продемонстрируем поиск пяти ближайших пользовательских пространств эмбеддингов к пространству пользователя с идентификатором `dd1457ebb737a8444bb9d942d1b6e4b7`:

In [ ]:
model_p2v = Pref2Vec(depth=500)
model_p2v.load_corpus(filepath_or_buffer = '/content/drive/MyDrive/ВКР/comics_graphic/children_corpus_pref2vec.csv')
most_similar = model_p2v.most_similar_users(
    df_user_data.loc['dd1457ebb737a8444bb9d942d1b6e4b7'],
    topn=5,
    show_info=True
)

Ближайщие пользователи к dd1457ebb737a8444bb9d942d1b6e4b7:
1) Пользователь №4b925decbd4de899d05bcf6285876127
   shelved - 0.9992012092128449
   rating_0 - 0.8964882483884777
   rating_1 - 0.9757238640172818
   rating_2 - 0.9570338527936179
2) Пользователь №16c0f7c0428bb7ad46451b5ec9f86b4b
   shelved - 0.9991744356527119
   read - 0.9996505076983649
   rating_0 - 0.8956383000000969
   rating_2 - 0.959148916027557
3) Пользователь №273c76615176de3c7c964bbb7213b126
   rating_0 - 0.8964842335665072
   rating_2 - 0.956037160027809
   rating_3 - 0.9979285673762591
   rating_5 - 0.9995351431841714
4) Пользователь №ae1ce5698724d031f837cb040d029dcb
   read - 0.9996434792456268
   rating_2 - 0.9557316124579966
   rating_3 - 0.9978176448351996
   rating_4 - 0.99931098028142
5) Пользователь №b9cd1d642c2a8d91ecc26879b887e6c7
   shelved - 0.9991498346233626
   rating_2 - 0.9557481369183477
   rating_3 - 0.9978481038067485
   rating_5 - 0.9995597752908856


В результате был получен следующий объект:

In [ ]:
most_similar

[{'user_id': '2c80415981b61c1050bc08dd1b6f5c8f',
  'appearances': [('rating_0', 0.9837419557962908),
   ('rating_3', 0.997353244320649)]},
 {'user_id': 'a87b42d0abefde4d368d58eff67ae1a2',
  'appearances': [('read', 0.9996584539124895),
   ('rating_4', 0.9991708975264934)]},
 {'user_id': '89419c628d615279f2f4b2cc83fba1d5',
  'appearances': [('rating_1', 0.9800258661296223),
   ('rating_3', 0.9972506447398124)]},
 {'user_id': '0e0427a41c277a3063cf73e2e52e5abf',
  'appearances': [('read', 0.9996295275728299),
   ('rating_4', 0.99919878461287)]},
 {'user_id': 'cc54992be4a5c52d034a863d669ef8c5',
  'appearances': [('read', 0.9996256528092547),
   ('rating_4', 0.9992294296537455)]},
 {'user_id': '75a11d10f4522f5d836d4350f657a41b',
  'appearances': [('rating_0', 0.9841379305522019),
   ('rating_3', 0.9971625754047598)]},
 {'user_id': 'dfb61c63fd3d9961705822225ecdca24',
  'appearances': [('read', 0.9996321693169773),
   ('rating_4', 0.9991639286411652)]},
 {'user_id': 'f872b65c838a346d61d70ccbe

Для каждого вида взаимодействия посредством метода `__most_similar_users_by_interaction()` вычисляются косинусные сходства между центром кластера текущего вида взаимодействия целевого пространства и центрами кластеров текущего вида взаимодействия всех остальных пространств. Все пространства корпуса ранжируются по рассчитанным значениям сходства, в дальнейших расчётах будут использоваться первые N элементов каждого вида взаимодействия, где значение N равно значению `self.depth`. Результаты вычислений сохраняются в датасете `self.df_most_similar_users`:

In [ ]:
model_p2v.df_most_similar_users.head()

,user_id,interaction,interaction_index,distance
0,c111cff4e87a47a6bd73d9f11c0f8a51,read,1,0.999361
1,bc7e18bade3dd9ed6b1c7e42d0b07130,read,2,0.999196
2,20da1218937deefcb3f8120c59c4b3ad,read,3,0.999105
3,a6b16ffba79873bf733e18f22999671d,read,4,0.999039
4,d3763802aec2055a40a1ea722890dbe3,read,5,0.998972


Данные поля `self.df_most_similar_users` группируются по значениям идентификаторов пользователей и агрегируются: вычисляется количество видов взаимодействий (столбец `interaction`), в рамках которых пользовательское пространство оказалось схожим с целевым, и среднее значение места в ранжированном списке в рамках вида взаимодействия (столбец `interaction_index`).

In [ ]:
df_most_similar_users_aggregated = model_p2v.df_most_similar_users.groupby('user_id').agg({'interaction': 'count', 'interaction_index': 'mean'})
df_most_similar_users_aggregated.head()

,interaction,interaction_index
user_id,,
001b6601f334a250d1d5012bd80f0e61,1,92.0
00b486e120b063e79f02f238e4c1e806,1,160.0
00d40ea21d1c012c796a7f913e290457,1,125.0
011e0fa5b4b74b97704bc897891dcbb4,1,206.0
012d567c99ae4f64f4ba533d96333efe,1,117.0


После датасет сортируется: по убываю относительно количества видов взаимодействий (столбец `interaction`), в рамках которых пользовательское пространство оказалось схожим с целевым, и по возрастанию относительно среднего значения места в ранжированном списке в рамках вида взаимодействия (столбец `interaction_index`). Выбираются первые topn элементов как ближайшие к целевому.

In [ ]:
df_most_similar_user_topn = df_most_similar_users_aggregated.sort_values(by=['interaction', 'interaction_index'], ascending=[False, True]).head(10)
df_most_similar_user_topn

,interaction,interaction_index
user_id,,
8f451697cae31799ae47afd86b2b5334,2,18.5
36c885627cb4289198a5bdbc2e68211c,2,114.0
bad4164e62cbb33a9613d87b47d1ffca,2,121.0
72c0616c598fc9573bccbf29c702397e,2,125.0
f7f4573a2dbcf4d88acdb51ef5d43f6f,2,145.0
db10048081045becfa1f9a9d82ada8f7,2,154.0
e960dc8bd1ed318b52ab318842e4167e,2,154.5
dc03926d9766ee1a7ff1078a71e02270,2,182.5
8d41c71675029f04674dd6d2ad42a5ea,2,205.0


## 6) Построение рекомендаций

Также приведём пример построения рекомендаций для пользователя с идентификатором `dd1457ebb737a8444bb9d942d1b6e4b7`:

In [ ]:
filename_corpus = '/content/drive/MyDrive/ВКР/comics_graphic/comics_graphic_corpus_pref2vec.csv'

model_p2v = Pref2Vec(depth=500)
model_p2v.load_corpus(filename_corpus)

recommendations = model_p2v.recommend_for_user(
    target_user_data=df_user_data.loc['dd1457ebb737a8444bb9d942d1b6e4b7'],
    topn=5,
    show_info=True
)

Рекомендации для пользователя dd1457ebb737a8444bb9d942d1b6e4b7:
1) Книга #18988096 - 0.9999803445024014
2) Книга #9565237 - 0.9999409446372801
3) Книга #10208216 - 0.9998653343176523
4) Книга #22839420 - 0.9998469434320814
5) Книга #22677665 - 0.9997069767643024


Также обратимся к датасету `books.csv` и выведем содержимое сформированных рекомендаций:

In [ ]:
books[books.index.isin([18988096, 9565237, 10208216, 22839420, 22677665])]

,description,title
book_id,,
10208216,Tak disangka FBI harus bertarung mati-matian m...,DETEKTIF CONAN vol. 59
22677665,"Ghastly beheadings, bloody murders, and cold-h...","Detective Conan, vol. 1"
18988096,"Qian Ru , Zhen Ye Zhong nooCha Hui !!\nMing Me...",黒執事 XVII [Kuroshitsuji XVII]
9565237,Seseorang yang misterius mengundang enam orang...,Detektif Conan Vol. 30
22839420,Huai saretaZhu woHu re----.\nRen Lang noZhou i...,黒執事 XX [Kuroshitsuji XX]


Чтобы проанализировать семантическую взаимосвязь результатов, полученных при построении рекомендаций, и целевого пространства, рассмотрим облака слов пространства эмбеддингов пользователя `dd1457ebb737a8444bb9d942d1b6e4b7`:

In [ ]:
ues = UES(df_user_data.loc['dd1457ebb737a8444bb9d942d1b6e4b7'])
ues.show_wordclouds()

Можем убедиться в том, что между результатами, полученными при построении рекомендаций, и целевым пространством эмбеддингов определённо присутствует взаимосвязь.